In [1]:
import torchvision
from torchvision import transforms
import torch
import matplotlib.pyplot as plt 
import numpy as np 
from skimage import io,transform,morphology
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')
import time

In [40]:
train_dir = 'data/chest_xray/train/'
test_dir = 'data/chest_xray/test/'


transform = transforms.Compose([
    transforms.ToTensor(), # pic-->tensor()
    transforms.Resize([128,128]),
    transforms.Grayscale(), # 灰度
])

train_ds = torchvision.datasets.ImageFolder(
    train_dir,
    transform=transform
)
test_ds = torchvision.datasets.ImageFolder(
    test_dir,
    transform=transform
)

BATCHSIZE = 16
train_dl = torch.utils.data.DataLoader(
    train_ds,
    batch_size=BATCHSIZE,
    shuffle=True
)

test_dl = torch.utils.data.DataLoader(
    test_ds,
    batch_size=BATCHSIZE
)

In [41]:
# imgs,labels = next(iter(test_dl))

In [39]:
# img_1 = imgs[0].squeeze()
# plt.imshow(img_1,cmap='gray')

In [28]:
class CnnModel(nn.Module):
    # 构建组件
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,5) # 默认移动窗口为1（stride=1）
        self.conv2 = nn.Conv2d(6,16,5)
        self.Linear1 = nn.Linear(16*29*29,128)
        self.Linear2 = nn.Linear(128,2)

    # 前向传播
    def forward(self,inputs):
        x = torch.max_pool2d(torch.relu(self.conv1(inputs)),2)
        x = torch.max_pool2d(torch.relu(self.conv2(x)),2)
        # 将数据展平为2维度
        x = x.view(-1,16*29*29)
        x = torch.relu(self.Linear1(x)) # 连接全连接
        x = self.Linear2(x) # 输出层
        return x

In [30]:
# models = CnnModel()
# models(imgs)

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 模拟模型放入加速运算的GPU
model = CnnModel().to(device)

In [32]:
# 定义一个损失函数(初始化损失函数)
loss_fn = nn.CrossEntropyLoss()
# 初始化优化器
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [33]:
# 搭建训练步骤
def train(dataloader,model,loss_fn,optimizer):
    # acc 当前样本一共有多少个
    size = len(dataloader.dataset)
    num_batcher = len(dataloader)
    # 初始化loss值
    train_loss,correct = 0,0
    for X,y in dataloader:
        X,y = X.to(device),y.to(device)
        # 前向传播（推理）
        pred = model(X)
        # 产生误差
        loss = loss_fn(pred,y) # 返回平均误差
        # 归零权重(固定写法)
        optimizer.zero_grad()
        # 反向传递
        loss.backward()
        # 利用梯度下降优化参数
        optimizer.step()
        # 放弃张量微分的追踪（小验证）
        with torch.no_grad():
            train_loss +=loss.item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    correct /= size
    train_loss /= num_batcher
    return train_loss,correct

In [34]:
# 搭建训练模块
def test(dataloader,model):
    # acc 当前样本一共有多少个
    size = len(dataloader.dataset)
    num_batcher = len(dataloader)
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            # 推理
            pred = model(X)
            test_loss += loss_fn(pred,y)
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    correct /= size
    test_loss /= num_batcher
    return test_loss,correct

In [2]:
epochs = 5
train_acc=[]
train_loss =[]
test_acc = []
test_loss = []
for epoch in range(epochs):   
    start = time.time()
    epoch_loss,epoch_acc = train(dataloader=train_dl,model=model,loss_fn=loss_fn,optimizer=optimizer)
    epoch_test_loss,epoch_test_acc = test(dataloader=test_dl,model=model)
    end = time.time()
    times = end-start
    train_acc.append(epoch_acc)
    train_loss.append(epoch_loss)
    test_acc.append(epoch_test_acc)
    test_loss.append(epoch_test_loss)
    print('训练epoch{},训练集损失值:{:.2f},训练集的准确率:{:.1f}%,测试集损失值:{:.2f},测试集的准确率:{:.2f}%,消耗时间：{:.2f}s'
          .format(epoch+1,epoch_loss,epoch_acc*100,epoch_test_loss,epoch_test_acc*100,times))